In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
#from libs.logger import Logger
import logging
from libs.s3 import S3
import pyodbc
from client_based_code.kickz_code import CONNECTION_STRING

logger = logging.getLogger(__name__)

from libs.logger import Logger
logger = Logger().get_print_logger()

In [ ]:
gs_path_token = './auth_files/token.pickle'
gs_path_client_secret = './auth_files/client_secret_708086849726-edgo6g4pigkf5rj0qc52rir18oso0kto.apps.googleusercontent.com.json'
gs_spreadsheet_id = '1PHzUwQEZ5gLmf32D3Akx_qL5iFta8HuCnZWtWVb3bTk'

from libs.google_sheets import GoogleSheetsApi

gapi = GoogleSheetsApi(
    path_token = gs_path_token,
    path_client_secret = gs_path_client_secret
)

In [ ]:
df_pts = gapi.google_sheet2df(gs_spreadsheet_id, 'products_to_score')

In [ ]:
SQL = 'SELECT TOP 10 *  FROM [rawone11].[v_pim_articles_data]'

In [ ]:
with pyodbc.connect(CONNECTION_STRING) as con:
    df = pd.read_sql(SQL, con)

In [ ]:
df_pts['date_added'] = pd.

In [ ]:
from client_based_code.kickz_code import S3ProductsToScore, get_orders

In [ ]:
#S3ProductsToScore.store(df_products_to_score)

In [ ]:
df_products_to_score = S3ProductsToScore.load_latest()

In [ ]:
import datetime as dt

In [ ]:
df_products_to_score['date_added'] = df_products_to_score['date_added'] - dt.timedelta(days=30)

In [ ]:
to_date = dt.date.today()
from_date = to_date - dt.timedelta(days=30)

In [ ]:
df_orders = get_orders(from_date, to_date)

In [ ]:
df_orders_grouped = df_orders.groupby(['brand','style'], as_index=False)['quantity'].sum()

In [ ]:
df_final = (
    df_products_to_score[['brand','style','product_name']]
    .merge(df_orders_grouped, on=['brand','style'], how='inner')
    .fillna(0)
    .sort_values('quantity',ascending=False)\
    .reset_index(drop=True)
)

In [ ]:
df_final

In [ ]:
from client_based_code.kickz_code import get_all_products, get_orders,get_google_ads_data,get_style_items_categories,get_quantities_from_inventory

In [ ]:
df = get_google_ads_data(dt.date(2025,9,1), dt.date.today())

In [ ]:
df

In [ ]:
df= get_quantities_from_inventory()

In [ ]:
df

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('backup/df_recommendations_20250922.parquet')

In [ ]:
df.groupby(['category','recom_change']).size()

In [ ]:
df[['style', 'country_code','impressions_demand','ctr_demand']]

In [ ]:
import datetime as dt 
from client_based_code.kickz_code import get_google_ads_data

def adjust_gads_data(df_gads):
    df_gads.loc[df_gads['clicks'] > df_gads['impressions'],'impressions'] = df_gads.loc[df_gads['clicks']> df_gads['impressions'],'clicks']
    df_gads['ctr'] = df_gads['clicks'].divide(df_gads['impressions']).fillna(0)

    return df_gads

today = dt.date.today()

last_week_start = today - dt.timedelta(7)
week_before_start = today - dt.timedelta(14)

df_gads_last_week = adjust_gads_data(get_google_ads_data(last_week_start))
df_gads_week_before = adjust_gads_data(get_google_ads_data(week_before_start, last_week_start))

df_gads_ratio = df_gads_last_week.merge(
    df_gads_week_before,
    left_on = ['country_code','style','brand'],
    right_on = ['country_code','style', 'brand'],
    suffixes = ('_last_week','_week_before')
    )\
     .assign(
        clicks_ratio = lambda x: x['clicks_last_week'] / x['clicks_week_before'],
        impresions_ratio = lambda x: x['impressions_last_week'] / x['impressions_week_before'],
        ctr_ratio =lambda x: x['ctr_last_week'] / x['ctr_week_before']
    )\
    .fillna(0)

df_gads_ratio = df_gads_ratio[['country_code','style','clicks_ratio', 'impresions_ratio', 'ctr_ratio']]


In [ ]:
df_gads_ratio[df_gads_ratio['style'].str.contains('5620')]

In [ ]:
df[['style', 'country_code']]

In [ ]:
df_gads_ratio.merge(df[['style', 'country_code']], on=['style', 'country_code'])

In [ ]:
df[df['style'] == 'fn2651-100'][['country_code', 'impressions_demand','ctr_demand']]

In [ ]:
import pandas as pd
import logging
import sentry_sdk
from sentry_sdk.integrations.logging import LoggingIntegration
from libs.logger import Logger
from update_prices import PricingLogic
from settings import kickz

In [ ]:
# logging
logger = Logger().get_full_logger(
    filename='./logs/update_prices.log',
    log_level=logging.INFO,
    print_level=logging.INFO
)

In [ ]:
updater = PricingLogic(settings=kickz)
#updater.run(insert_into_production=False, insert_into_s3=False)

In [ ]:
updater._load_conversion_rates()

In [ ]:
df_recommendations = pd.read_parquet('backup/df_recommendations_20250929.parquet')

In [ ]:
df_recommendations

In [ ]:
df_recommendations = df_recommendations[
    (~df_recommendations['recom_price'].isnull())
  & (~df_recommendations['base_price'].isnull())
  & (df_recommendations['recom_change'] != 'NOT ENOUGH DATA')
  & (df_recommendations['master_switch'] == 1)
]

# CHECK NA CHYBNE CENY 
MAX_DISCOUNT = 0.1 # 1 - maximalna mozna zlava ktoru je mozne dat na produkt 
df_recommendations = df_recommendations[
    (df_recommendations['recom_price'].between(df_recommendations['base_price']*MAX_DISCOUNT, df_recommendations['base_price']))
  & (df_recommendations['base_price'] > 0)
]

# currency mapper
df_recommendations['currency'] = df_recommendations['country_code'].apply(lambda country_code: COUNTRY_CODE_CURRENCY_MAPPER[country_code])

In [ ]:
df_recommendations

In [ ]:
df_recommendations.apply(lambda row: row['recom_price'] * self.conversion_rates[row['currency']], axis=1)

In [ ]:
df_recommendations['recom_price_local_currency'] = df_recommendations.apply(lambda row: row['recom_price'] * self.conversion_rates[row['currency']], axis=1)

In [ ]:
df_recommendations = updater.df_recommendations

In [ ]:
df_recommendations['our_min_possible_price'] = df_recommendations['base_price'] * (1 - df_recommendations['max_discount'])
df_recommendations['our_max_possible_price'] = df_recommendations['base_price'] * (1 - df_recommendations['min_discount'])

In [ ]:
df_imp = df_recommendations[df_recommendations['category'] != 'ST']
df_st = df_recommendations[df_recommendations['category'] == 'ST'].query('country_code == "DE"')

In [ ]:
df_imp[['date','brand','style', 'country_code','base_price','price','recom_price','recom_change',
        'min_discount', 'max_discount','impressions_demand', 'ctr_demand','total_demand',
        'sold_items_7_days','our_min_possible_price','our_max_possible_price']]

In [ ]:
from client_based_code.kickz_code import get_google_ads_data,get_quantities_from_inventory
import datetime as dt

In [ ]:
df_gads = get_google_ads_data(from_date=dt.date(2025,9,16))

In [ ]:
df_gapi_714_ratios = updater.df_gapi_714_ratios

In [ ]:
df_gapi_714_ratios[df_gapi_714_ratios['style'] == 'ij5620']

In [ ]:
df_gads[df_gads['style'].str.contains('ij5620')]

In [ ]:
df_st[['date','brand','style', 'country_code','base_price','price','recom_price','recom_change',
        'min_discount', 'max_discount','sell_power_day','sell_power_week','is_new_product','last_day_sell_power_week','ST_setting','item_category',
       'season_length','days_from_season_start',
      'sold_items_day','sold_items_season','quantity_in_inventory']].dropna(subset=['base_price'])

In [ ]:
updater.discount_levels['ST'][('adidas originals','DE')]

In [ ]:
updater.st_settings

In [ ]:
df_quantity_in_inventory = get_quantities_from_inventory()

In [ ]:
df_quantity_in_inventory[df_quantity_in_inventory['style'] == 'ig6213']

In [ ]:
updater.quantities_in_inventory

In [ ]:
import pandas as pd

In [ ]:
df_recommendations = pd.read_parquet('backup/df_recommendations_20250924.parquet')

In [ ]:
df_recommendations[['brand','style','style_count_all_competitors', 'style_count_important_competitors',
       'style_all_competitors_list', 'style_all_competitors_links',
       'style_all_competitors_prices', 'style_all_competitors_in_stock',
       'style_all_competitors_price_change_day',
       'style_important_competitors_list', 'style_important_competitors_links',
       'style_important_competitors_prices',
       'style_important_competitors_in_stock',
       'style_important_competitors_price_change_day']].sort_values('style_count_all_competitors')

In [ ]:
df_recommendations.sort_values('style_count_all_competitors')

In [ ]:
df_top_100  = df_recommendations.groupby(['brand','style'], as_index=False)[['sold_items_14_days']].sum().sort_values('sold_items_14_days', ascending=False).head(100)

In [ ]:
df_recommendations['our_min_possible_price'] = df_recommendations['base_price'] * (1 - df_recommendations['max_discount'])
df_recommendations['our_max_possible_price'] = df_recommendations['base_price'] * (1 - df_recommendations['min_discount'])

In [ ]:
df_imp = df_recommendations[df_recommendations['category'] != 'ST']
df_st = df_recommendations[df_recommendations['category'] == 'ST'].query('country_code == "DE"')

In [ ]:
df_imp[['date','brand','style', 'country_code',
        'base_price','price','recom_price','recom_change',
        'min_discount', 'max_discount','impressions_demand', 'ctr_demand','total_demand',
        'sold_items_7_days','our_min_possible_price','our_max_possible_price']]

In [ ]:
df_st[['date','brand','style', 'country_code','base_price','price','recom_price','recom_change',
        'min_discount', 'max_discount','sell_power_day','sell_power_week','is_new_product','last_day_sell_power_week','ST_setting','item_category',
       'season_length','days_from_season_start',
      'sold_items_day','sold_items_season','quantity_in_inventory']].dropna(subset=['base_price'])['recom_change'].value_counts()

In [ ]:
df_sample = df_recommendations.merge(df_top_100[['brand','style']], on=['brand','style'])

In [ ]:
df_sample = df_sample[
    (~df_sample['recom_price'].isnull())
  & (~df_sample['base_price'].isnull())
  & (df_sample['recom_change'] != 'NOT ENOUGH DATA')
  & (df_sample['master_switch'] == 1)
]

In [ ]:
export_cols = [
    'date',
    'brand',
    'product_name', 
    'style',
    'country_code',
    'category' ,
    'price',
    'base_price',
    'recom_price',
    'our_min_possible_price',
    'our_max_possible_price',
]
df_sample['date'] = pd.to_datetime(df_sample['date']).dt.date
df_sample[export_cols].round(2).to_csv('kickz_results.csv', index=False)

In [ ]:
df_sample['date']

In [ ]:
from client_based_code.kickz_code import load_prices

In [ ]:
df_prices = load_prices()

In [ ]:
df_prices[df_prices['style'] == 'hq8707']

In [ ]:
df_prices['price_local'].round()

In [ ]:
round(5.10)

In [ ]:
from client_based_code.kickz_code import load_competitors_data
import datetime as dt

In [ ]:
from libs.help_functions import (
    COUNTRY_CODE_CURRENCY_MAPPER,
    is_our_shop,
    countryCompetitors2dict,
    clean_country_competitors,
    is_important_competitor
)

In [ ]:
from_date = dt.date.today() - dt.timedelta(days=3)
to_date = dt.date.today()

In [ ]:
import json

with open("auth_files/eas-core-34afc8b13f75.json", "r") as f:
    credentials = json.load(f)

In [ ]:
df = load_competitors_data(credentials, from_date, to_date, 90)

In [ ]:
df

In [ ]:
df['currency'] = df.apply(
    lambda row: COUNTRY_CODE_CURRENCY_MAPPER[row['country_code']] 
    if row['currency'] == '' 
    else row['currency'],
    axis=1
)

In [ ]:
df['is_our_shop'] = df.apply(
    lambda row: is_our_shop(row['url'], row['competitor_shop_name'],['kickz']), 
    axis=1
)

In [ ]:
df['is_important_competitor'] = df.apply(
    lambda row: is_important_competitor(row, country_competitors, 90),
    axis=1
)

In [ ]:
df[df['is_important_competitor']]

In [ ]:
df_competitors = gapi.google_sheet2df(gs_spreadsheet_id, 'relevant_competitors')
        
# dictionary
country_competitors = countryCompetitors2dict(df_competitors)

# Ocisti konkurenciu
# 'store.nike.com' => 'nike'
# 'https://www.runningpro.sk/' => 'runningpro'
# ...
country_competitors = clean_country_competitors(country_competitors)

In [ ]:
country_competitors

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
df = pd.read_parquet('s3://autopricing/kickz/products_to_score/20250925210451.parquet')

In [ ]:
pd.to_datetime(df['date_added'], dayfirst=False)

In [ ]:
df = pd.read_parquet('ap_export.parquet')

In [ ]:
SQL = """
SELECT      
        LOWER(TRIM(brand)) AS brand,
        LOWER(TRIM(style_id)) AS style,
        TRIM(material_number) AS material_number
FROM [rawone11].[v_pim_articles_data]
WHERE item_shop_active_kickz = 1
    AND name IS NOT NULL
    AND style_id IS NOT NULL
    AND brand IS NOT NULL;
"""

In [ ]:
with pyodbc.connect(CONNECTION_STRING) as con:
    df_mapper = pd.read_sql(SQL, con)

In [ ]:
x = df.merge(df_mapper, on=['brand','style'])

In [ ]:
x['export_country_code'] = 'kickz-' + x['country_code']
x['export_status'] = 'DISCOUNT'
x['export_from_date'] = x['date'].dt.strftime("%Y-%m-%dT%H:00:00")
x['export_to_date'] = pd.to_datetime(dt.datetime(2100,1,1)).strftime("%Y-%m-%dT%H:00:00")

In [ ]:
df_export = x[['material_number', 'recom_price_local_currency','export_status','export_from_date','export_to_date','export_country_code','currency']]

In [ ]:
df_export

In [ ]:
csv_data = df_export.head(10).to_csv(index=False, header=False, sep=';')


In [ ]:
csv_data

In [ ]:
def generate_filename(prefix="master/hotfolder/kickz/prices"):
    # current UTC time
    now = dt.datetime.now(dt.timezone.utc)
    
    # Unix timestamp in milliseconds
    ts_millis = int(now.timestamp() * 1000)
    
    # Human-readable timestamp (ISO-like) with milliseconds and Z
    dt_str = now.strftime("%Y-%m-%dT%H-%M-%S.%f")[:-3] + "Z"
    
    # Build filename
    filename = f"{prefix}-{ts_millis}.csv.{dt_str}"
    
    return filename

# Example usage
blob_name = generate_filename()

In [ ]:
blob_name

In [ ]:
connection_string = (
    "DefaultEndpointsProtocol=https;"
    "AccountName=o3w4n3fotw789gttys79amu;"
    "AccountKey=ltkXAjB76EPhkIfKM9Drmnn9k8XP8Zvb4d5ymrmQCGNbmFWeHHYGZvtB7VJuwnRiZHc64vN62cdO+AStQeCLVg==;"
    "EndpointSuffix=core.windows.net"
)
container_name = "hybris"


# Create blob client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload CSV
blob_client.upload_blob(csv_data, overwrite=True)
print("✅ DataFrame uploaded successfully as CSV (no header).")

In [ ]:
export_blob_name = "master/hotfolder/kickz/prices-{ts_millis}.csv"

In [ ]:
export_blob_name.format(ts_millis='dsdsd')

In [1]:
import pandas as pd
import settings.kickz as settings
from libs.help_functions import upload_dataframe_to_azure_blob_storage
from client_based_code.kickz_code import load_material_number_mapper,load_prices

In [ ]:
a = load_prices()

In [ ]:
a

In [2]:
df = pd.read_parquet('ap_export.parquet')

In [16]:
df = df[df['material_number'] != 'Not Defined']

In [22]:
df[df['material_number'] == 'IJ5626-M']

,material_number,recom_price_local_currency,export_status,export_from_date,export_to_date,export_country_code,currency
1,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-DE,EUR
7,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-ES,EUR
13,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-FR,EUR
19,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-IT,EUR
25,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-AT,EUR
31,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-NL,EUR
37,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-BE,EUR
43,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-DK,EUR
49,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-FI,EUR
55,IJ5626-M,22.95,ORIGINAL,2025-10-08T13:00:00,2100-01-01T00:00:00,kickz-NO,EUR


In [ ]:
import datetime as dt

In [ ]:
df['export_country_code'].unique()

In [ ]:
df.head(100)['export_country_code'].unique()

In [ ]:
upload_dataframe_to_azure_blob_storage(
    df.head(10000),
    settings.export_container_name,
    settings.export_blob_name.format(ts_millis=int(dt.datetime.now(dt.timezone.utc).timestamp() * 1000)),
    settings.export_connection_string
)

In [ ]:
df.iloc[72:73]

In [ ]:
df.head(75)

In [ ]:
from client_based_code.kickz_code import load_material_number_mapper

In [ ]:
df_mn = load_material_number_mapper()

In [ ]:
df_mn[df_mn['material_number'] == 'IJ5626-K2XL']

In [ ]:
df2 = pd.read_csv('prices-1759372237007.csv')

In [ ]:
df

In [ ]:
df2.iloc[71:72]